# Import necessary libraries

In [ ]:
import csv
import nltk
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report 
from sklearn.feature_selection import VarianceThreshold
import json
import copy
import pickle

# Load data from .csv file

In [ ]:
category = []
content = []

with open('D:/PythonProject/News-classfication/Preprocessing/output.csv', 'r', encoding='utf-8') as file:
    csvreader = csv.reader(file)
    fields = next(csvreader)
    for row in csvreader:
        category.append(row[0])
        content.append(str(row[1]))

#print(content)

# Preprocessing
Convert labels into numeric values

In [ ]:
encoder = LabelEncoder()
Y = encoder.fit_transform(category)

#for i in range(len(category)):
#   print(category[i], Y[i])


# Split data into train, test and validation set
X_train, X_test, Y_train, Y_test = train_test_split(content, Y)
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train)
print(len(X_train))
print(len(X_validation))
print(len(X_test))

Split the data

## TfIdf-Vectorize data
Tf-Idf stands for Term Frequency-Inverse Document Frequency. It is a techinque to quantify a word in documents, we generally compute a weight to each word which signifies the importance of the word which signifies the importance of the word in the document and corpus.

In [ ]:
"""tfidf_vectorizer = TfidfVectorizer(analyzer='word',stop_words='english', max_df=0.8, min_df=3, sublinear_tf=True)

tfidf_vectorizer.fit(X_train)
X_train = tfidf_vectorizer.transform(X_train)
X_validation = tfidf_vectorizer.transform(X_validation)
X_test = tfidf_vectorizer.transform(X_test)
print(X_train)
"""

tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
stop_words = nltk.corpus.stopwords.words("english")
#print(stop_words)
vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize, stop_words=stop_words)
print(vectorizer)

vectorizer.fit(iter(X_train))
X_train = vectorizer.transform(iter(X_train))
X_validation = vectorizer.transform(iter(X_validation))
X_test = vectorizer.transform(iter(X_test))
print(X_train[2])

# Feature Reduction

In [ ]:
print("Number of features before reduction : ", X_train.shape[1])
selection = VarianceThreshold(threshold=0.001)
Xtr_whole = copy.deepcopy(X_train)
Ytr_whole = copy.deepcopy(Y_train)
selection.fit(X_train)
X_train = selection.transform(X_train)
X_validation = selection.transform(X_validation)
X_test = selection.transform(X_test)
print("Number of features after reduction : ", X_train.shape[1])

# Training model

In [ ]:
svm_clf = SVC()
svm_clf.fit(X_train, Y_train)
prediction = svm_clf.predict(X_validation)

#danh gia hieu nang

print(classification_report(Y_validation, prediction, target_names=encoder.classes_))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, Y_train)
prediction = nb.predict(X_validation)

print(classification_report(Y_validation, prediction, target_names=encoder.classes_))

# Saving model

In [ ]:
with open('svm_model.pkl', 'wb') as file:
    pickle.dump(svm_clf, file)

# Load model and predict

In [ ]:
file = open('D:/PythonProject/News-classfication/svm_model.pkl', 'rb+')
clf = pickle.load(file)

test_prediction = svm_clf.predict(X_validation)
print(classification_report(Y_validation, prediction, target_names=encoder.classes_))


# Report on test set

In [ ]:
predict = svm_clf.predict(X_validation)
print(classification_report(Y_validation, predict, target_names=encoder.classes_))


# Predicting a single headline

In [ ]:
from Preprocessing.xxx import preprocess

# An example 
title = "A man has been killed, polices have no way to find the crime"
title = preprocess(title)
titles = []
titles.append(title)
titles = vectorizer.transform(titles)
titles = selection.transform(titles)

predict = svm_clf.predict(titles)
print(encoder.inverse_transform(predict))